<a href="https://colab.research.google.com/github/BrainyHaron/DigitalBreakthrough/blob/Brainy's_Projects/das_pirat_OS_vol_3(fin).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install nltk
!pip install scipy

In [4]:
import numpy as np
from nltk.stem import *
from nltk.stem.porter import *
stemmer = PorterStemmer()
import os
import json
import pandas as pd
from google.colab import drive
# из библиотеки google.colab импортируем класс files
from google.colab import files
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [5]:
from google.colab import drive
drive.mount('/content/drive')
path1='/content/drive/MyDrive/das_pirat_OS/all'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import json
path_all =path1
full_list3=[]
i=0
for file in os.listdir(path_all):
    try:
        with open(path_all+"/"+file,'r', encoding='Utf-8') as json_data:
            res_loaded = json.load(json_data)
            #k=res1_loaded['question']


            answer = res_loaded['answers'] #зашиваем список со  всеми словарями наших постов
            full_list = {'questions':[],'id':[],'answers':[]}
            full_list2={}
            full_list = {'questions':[],'id':[],'answers':[]}
            question_list={}
            id_list={}
            for i in range(0,len(answer)):
                full_list1=[]
                question_list['questions']= res_loaded['question']
                question_list['id']= res_loaded['id']
                for j in range(len(answer)):

                    full_list1.append(answer[j])
                full_list2['answers']=full_list1


            full_list.update(full_list2)
            full_list.update(question_list)
            full_list.update(id_list)
            id1=full_list['id']
            k1=full_list['questions']
            for answer in full_list['answers']:
                l = [k1]+[id1]+[answer['answer'], answer['count']]
                full_list3.append(l)

    except ValueError:
        pass


print(full_list3[0:5])

[['Что вы сможете использовать в своей работе?', 10106, 'формулировать проблему, 5 поче', 1], ['Что вы сможете использовать в своей работе?', 10106, 'поиск потерь', 1], ['Что вы сможете использовать в своей работе?', 10106, 'правило 5 «почему» :)', 1], ['Что вы сможете использовать в своей работе?', 10106, 'умение работать в команде', 1], ['Что вы сможете использовать в своей работе?', 10106, 'картирование и анализ', 1]]


In [7]:
dat = pd.DataFrame(full_list3)
pd.set_option('display.max_colwidth', None)
dat.columns = ['question','id', 'answer', 'count']
dat

,question,id,answer,count
0,Что вы сможете использовать в своей работе?,10106,"формулировать проблему, 5 поче",1
1,Что вы сможете использовать в своей работе?,10106,поиск потерь,1
2,Что вы сможете использовать в своей работе?,10106,правило 5 «почему» :),1
3,Что вы сможете использовать в своей работе?,10106,умение работать в команде,1
4,Что вы сможете использовать в своей работе?,10106,картирование и анализ,1
...,...,...,...,...
29659,Что поможет сделать программу более полезной для вас?,6343,больше практики,1
29660,Что поможет сделать программу более полезной для вас?,6343,ничего. все отлично,1
29661,Что поможет сделать программу более полезной для вас?,6343,все хорошо,1
29662,Что поможет сделать программу более полезной для вас?,6343,офлайн формат,1


In [8]:
dat['merged_q_a']=dat['question']+' '+dat['answer']
dat.head()

,question,id,answer,count,merged_q_a
0,Что вы сможете использовать в своей работе?,10106,"формулировать проблему, 5 поче",1,"Что вы сможете использовать в своей работе? формулировать проблему, 5 поче"
1,Что вы сможете использовать в своей работе?,10106,поиск потерь,1,Что вы сможете использовать в своей работе? поиск потерь
2,Что вы сможете использовать в своей работе?,10106,правило 5 «почему» :),1,Что вы сможете использовать в своей работе? правило 5 «почему» :)
3,Что вы сможете использовать в своей работе?,10106,умение работать в команде,1,Что вы сможете использовать в своей работе? умение работать в команде
4,Что вы сможете использовать в своей работе?,10106,картирование и анализ,1,Что вы сможете использовать в своей работе? картирование и анализ


In [9]:
col_list = dat["merged_q_a"].values.tolist()
print(col_list[100])

Ваши открытия за время обучения на модуле не думал, что на квадратный ме


In [10]:
class LSI(object):
	def __init__(self, stopwords, ignorechars, docs):
		self.wdict = {}
		self.dictionary = []
		self.docs = []
		self.stopwords = stopwords
		if type(ignorechars) == str: ignorechars = ignorechars.encode('utf-8')
		self.ignorechars = ignorechars
		for doc in docs: self.add_doc(doc)

	def prepare(self):
		self.build()
		self.calc()

	def dic(self, word, add = False):
		if type(word) == str: word = word.encode('utf-8')
		# чистим от лишних символом
		word = word.lower().translate(None, self.ignorechars)
		word = word.decode('utf-8')
		# приводим к начальной форме
		word = stemmer.stem(word)
		# если слово есть в словаре возвращаем его номер
		if word in self.dictionary: return self.dictionary.index(word)
		else:
			# если нет и стоит флаг автоматически добавлять то пополняем словари возвращвем код слова
			if add:
				#self.ready = False
				self.dictionary.append(word)
				return len(self.dictionary) - 1
			else:
				return None


	def add_doc(self, doc):
		words = [self.dic(word, True) for word in doc.lower().split()]
		self.docs.append(words)
		for word in words:
			if word in self.stopwords:
				continue
			elif word in self.wdict:
				self.wdict[word].append(len(self.docs) - 1)
			else:
				self.wdict[word] = [len(self.docs) - 1]

	def build(self):
      # убираем одиночные слова
		self.keys = [k for k in self.wdict.keys() if len(self.wdict[k]) > 0]
		self.keys.sort()
      # создаём пустую матрицу
		self.A = np.zeros([len(self.keys), len(self.docs)])
      # наполняем эту матрицу
		for i, k in enumerate(self.keys):
			for d in self.wdict[k]:
				self.A[i,d] += 1

#Вычисление U, S Vt - матриц
	def calc(self):
		self.U, self.S, self.Vt = np.linalg.svd(self.A)


	def TFIDF(self):
      # всего кол-во слов на документ
		wordsPerDoc = sum(self.A, axis=0)
      # сколько документов приходится на слово
		docsPerWord = sum(np.asarray(self.A > 0, 'i'), axis=1)
		rows, cols = self.A.shape
		for i in range(rows):
			for j in range(cols):
				self.A[i,j] = (self.A[i,j] / wordsPerDoc[j]) * np.log(float(cols) / docsPerWord[i])

	def dump_src(self):
		self.prepare()
		print ('Здесь представлен расчет матрицы ')
		for i, row in enumerate(self.A):
			print (self.dictionary[i], row)

	def print_svd(self):
		self.prepare()
		print ('Здесь сингулярные значения')
		print (self.S)
		print ('Здесь первые 3 колонки U матрица ')
		for i, row in enumerate(self.U):
			print (self.dictionary[self.keys[i]], row[0:3])
		print ('Здесь первые 3 строчки Vt матрица')
		print (-1*self.Vt[0:3, :])

	def find(self, word):
		self.prepare()
		idx = self.dic(word)
		if not idx:
			print('слово не встречается')
			return []
		if not idx in self.keys:
			print('слово отброшено как не имеющее значения которое через stopwords')
			return []
		idx = self.keys.index(idx)
		print('word --- '), word, '=', self.dictionary[self.keys[idx]], '.\n'
		# получаем координаты слова
		wx, wy = (-1 * self.U[:, 1:3])[idx]
		print(('word{}\t{:0.2f}\t{:0.2f}\t{}\n').format(idx, wx, wy, word)) #:0.2f
		arts = []
		xx, yy = -1 * self.Vt[1:3, :]
		for k, v in enumerate(self.docs):
			# получаем координаты документа
			ax, ay = xx[k], yy[k]
			#вычисляем расстояние между словом и документом
			dx, dy = float(wx - ax), float(wy - ay)
			arts.append((k, v, ax, ay, np.sqrt(dx * dx + dy * dy)))
		# возвращаем отсортированный по расстоянию список
		return sorted(arts, key = lambda a: a[4])


In [17]:

#path = '/content/drive/MyDrive/das_pirat_OS/labeled'
#full_list3=[]

docs = col_list [:100]
word = "проект"
ignorechars = ''',:'!'''
lsa = LSI([], ignorechars, docs)
lsa.build()
lsa.dump_src()
lsa.calc()
lsa.print_svd()

for res in lsa.find(word):
	print (res[0], res[4], res[1], docs[res[0]])

Здесь представлен расчет матрицы 
что [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
вы [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
сможете [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
использовать [1. 1. 1. 1. 1. 1. 1